# Bee Smart
 The objective of this project is to make a predictive model using AI about a bee behavoir during the day to detect anomalies and therefore know what's the cause in a pursuit to preserve this animals. 

 The idea for the data is classify them in 4 categories
1. BTE (by the entry) - short intervals of time, from 1 sec up to 3 min in where the bee is in the entrance (_piquera_) or over the reader (max of 30 min).
2. SM (short mission)- short activity between 3 - 6 min.
3. FG (foraging) - forage and exploration activity, from 6 minutes up to 6 hours. 
4. DB (departed bee) - last detection of every bee during its life

The data is in two main sets, one already labeled and the other one in raw

In the classification we consider to acitvity hours categories: 
1. Sunrise (7am - 12pm)
2. Sunset (12pm- 7pm)

 

    *Papers about similar projects in the past can be found at the end of this notebook.*



## 1. Raw data exploration and filtering
First we download the csv and filter the data, we have already a labeled archive but we're going to try to labeled by our own and compare results later. 


In [10]:
import pandas as pd

# Set the path for files
general_path = '/Users/ralo/Library/CloudStorage/OneDrive-UNIVERSIDADNACIONALAUTÓNOMADEMÉXICO/TECNOLOGÍA/' # General path for data files
filtered_data_path = 'Semestre 7/BeeSmart/data/filtered_data_sensors.csv'
raw_copy_MH1_path = 'Semestre 7/BeeSmart/data/raw_copy_MH1Exploration.xlsx'
raw_copy_MH2_path = 'Semestre 7/BeeSmart/data/raw_copy_MH2Exploration.xlsx'

# load them as datasets
df_filtered = pd.read_csv(general_path + filtered_data_path)
df_raw_MH1 = pd.read_excel(general_path + raw_copy_MH1_path, sheet_name=None) 
""" We add the sheetname=none to add all the sheets of our excel document, its important to notice that this creates a dictionary of dataframes not a single
dataframe, therefore if we want to acces to every sheet we use the key, either the index or the sheet name"""
df_raw_MH2 = pd.read_excel(general_path + raw_copy_MH2_path, sheet_name=None)

/opt/anaconda3/envs/bee_smart/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/opt/anaconda3/envs/bee_smart/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/opt/anaconda3/envs/bee_smart/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/opt/anaconda3/envs/bee_smart/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/opt/anaconda3/envs/bee_smart/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/opt/anaco

Watching the column names for our filtered dataset

As we mentioned before, this dataset is already filtered and labeled so no further filtering is required

In [11]:
df_filtered.head()

,Hive,Week,fecha,id,horario,behav,minutes,logmin
0,H2,W1,01/07/16,10,sunset,DB,0.016667,-1.778151
1,H2,W1,01/07/16,2,sunset,BTE,0.016667,-1.778151
2,H2,W1,01/07/16,2,sunset,BTE,0.016667,-1.778151
3,H2,W1,01/07/16,2,sunset,DB,0.016667,-1.778151
4,H2,W1,01/07/16,2,sunset,DB,0.016667,-1.778151


Watching the name of our sheets in the raw data

In [12]:
print(df_raw_MH1.keys())

dict_keys(['1Jul16', '2Jul16', '3Jul16', '4Jul16', '5Jul16', '6Jul16', '7Jul16', '8Jul16', '9Jul16', '10Jul16', '11Jul16', '12Jul16', '13Jul16', '14Jul16', '15Jul16', '16Jul16', '17Jul16', '18Jul16', '19Jul16', '20Jul16', '21Jul16', '22Jul16', '23Jul16', '24Jul16', '25Jul16', '26Jul16', '27Jul16', '28Jul16', '29Jul16', '30Jul16', '31Jul16', '1Ago16', '2Ago16', '3Ago16', '4Ago16', '5Ago16', '6Ago16', '7Ago16', '8Ago16', '9Ago16', '10Ago16', '11Ago16', '12Ago16', '13Ago16', '14Ago16', '15Ago16', '16Ago16', '17Ago16', '18Ago16', '19Ago16', '20Ago16', '21Ago16', '22Ago16', '23Ago16', '24Ago16', '25Ago16', '26Ago16'])


With a manual check we saw that for raw MH1 we have one excel sheet for every day, but not all days have data, with that knowledge lets delete them

In [13]:
# Converting keys to a list to acces them by index
sheet_names = list(df_raw_MH1.keys())

# Indexes to drop (days without data) (July 1, 4, 5, 6, 7)
drop = [0, 3, 4, 5, 6]

# Drop 
for idx in drop: 
    sheet = sheet_names[idx]
    df_raw_MH1.pop(sheet, None)



Now that we've filtered all the days without data we can concat all other sheets to create a unique dataframe

In [14]:
df_raw_MH1 = pd.concat(df_raw_MH1.values(), ignore_index=True)
df_raw_MH1.head()

,# bee_detection_data v_1_1_3,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Colmena,ID,Tiempo,Horam,HoraS,Horadecimal,Hora MX(-5),Hora_negativo,Hora_entero,Minutos,Segundos,Duracion (hr),Duracion(min)
0,20160702T171250Z,870020100100104300000023,7358E280113020002453172E,1530U2127,Edison-fzed437d00cex501,NaN,1,23,171250,0.200000,0.013889,17.213889,12.213889,NaN,12,12,50,NaN,NaN
1,20160703T152446Z,870020100100104300000004,27DDE2801130200028A017CB,1530U2127,Edison-fzed437d00cex501,NaN,1,4,152446,0.400000,0.012778,15.412778,10.412778,NaN,10,24,46,NaN,NaN
2,20160708T152037Z,870020100100104300000064,4B7BE2801130200024D0162D,1530U2127,Edison-fzed437d00cex501,NaN,1,064,152037,0.333333,0.010278,15.343611,10.343611,NaN,10,20,37,NaN,NaN
3,20160708T154856Z,870020100100104300000093,D483E28011302000205B17A2,1530U2127,Edison-fzed437d00cex501,NaN,1,093,154856,0.800000,0.015556,15.815556,10.815556,NaN,10,48,56,NaN,NaN
4,20160708T155529Z,FFFFFFFFFFFFFFFFFFFFFFFF,CE8CE28011302000205F17DE,1530U2127,Edison-fzed437d00cex501,NaN,F,FFF,155529,0.916667,0.008056,15.924722,10.924722,NaN,10,55,29,NaN,NaN


# Research papers and Thanks
I want to thank PhD Ulises Olivares for giving me the opportunity of being part of this project. 

Below you can find the different papers he have done about this topic if you want to read more about. 

https://doi.org/10.21203/rs.3.rs-6825046/v1

https://papers.ssrn.com/sol3/papers.cfm?abstract_id=4525546